# **Modeling and Evaluation Regression Predict Wing Span Notebook**

## Objectives

*   Fit and evaluate a regression model to predict Wing Span for Single piston Engined airplanes.


## Inputs

* outputs/datasets/collection/airplane_performance_study.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in their respective notebooks.

## Outputs

* feature_importance (3 files) 
* errors_analysis
* Predicted vs Actual Plot
* Residuals Distribution Plot
* Residuals vs Fitted Plot
* Train set
* Test set
* Modeling pipeline

---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Imports

In [4]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Load Data

In order to not compare apples with oranges the code below filters out all Multi Engined, jet and propjet leaving only the subset of **single Piston Engines**. It drops these columns as well as the performance modification (TP mods) column and the Hmax_(One) and ROC_(One)-columns since they only pertain to Multi Engined airplane.

In [ ]:
df = (pd.read_csv("outputs/datasets/collection/airplane_performance_study.csv")
      .query("Engine_Type == 0 and Multi_Engine == 0 and TP_mods == 0")  # subset of airplanes with single Piston Enginees without any Turbo Prop performance modification TP_mods dmultiple Engines
      .drop(labels=['Model', 'Company', 'THR', 'SHP', 'Engine_Type', 'Multi_Engine', 'TP_mods', 'Hmax_(One)', 'ROC_(One)'], axis=1)
      )

print(df.shape)
df.head(10)

---

## Define features and target

Define features and target

X are the features for the model (all columns in df except for target Wing_Span.
y is the target variable which is the Wing_Span column in the df.

In [6]:
# Define features and target
X = df.drop('Wing_Span', axis=1)  # Features (where I drop the target variable)
y = df['Wing_Span']  # Target variable

## Split Train Test Set

80% of df is split into the train set and 20% is left for the test set.
The random_state=0 parameter ensures reproducibility of the split.

We see that we have a suffiecent numper of data points to work with.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  # 80% to train and 20% to test.

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

---

# MP Pipeline: Regressor

## Create Pipeline

'scaler' applies standard scaling to the (numerical) features, ensuring that they have a mean = 0 and a standard deviation = 1.
'regressor' initializes Random Forest regression model for predicting target variable (Wing_Span).

In [8]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),  # Scale numerical features
    ('regressor', RandomForestRegressor(random_state=0))  # Regression model using Random Forrest
])

## Fit the model

build the training model using the training dataset.

StandardScaler is applied to scale the features in X_train.

RandomForestRegressor is trained on the scaled features (from previous step) and corresponding target values.

In [ ]:
pipeline.fit(X_train, y_train)

## Make predictions

Scaling test features using StandardScaler fitted during training.

Using the trained RandomForestRegressor to predict on scaled test data.

Output: The predicted values for target (Wing_Span) stored in the predictions variable.

In [10]:
predictions = pipeline.predict(X_test)

---

## Evaluate Model Test Set

### Mean Squared Error Calculation

The mean_squared_error computes the mean squared error (MSE) between the actual values (y_test) and the predicted values (predictions). The MSE provids a quantitative measure of the model's performance on the test dataset.

 MSE is ok but not great, typically off by almost 3 ft (square root on MSE)! However we will do a cross-validation downstreams to see if this more reliable value is better.

In [ ]:
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

## Print feature importances (for RandomForestRegressor)

feature_importance_df is created that in descending order (by importance) list an "importance score"-for each feature providing insights into which features are most influential in predicting the target variable (Wing_Span).It makes much sense that the weights are on top since the Wing Span is related to the Wing Area (not in the data set) and that Wing Area is directly related to lift that in turn is directly related to the weights (since the lift need to offset the weight in order for the airplane to fly)

In [ ]:
importances = pipeline.named_steps['regressor'].feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

Save above output

In [ ]:
# Save the feature names as a .pkl file
joblib.dump(feature_names.tolist(), 'outputs/ml_pipeline/predict_analysis/feature_names.pkl')

# Save the feature importance DataFrame as a .pkl file
joblib.dump(feature_importance_df, 'outputs/ml_pipeline/predict_analysis/feature_importance_df.pkl')


---

## Plot Regression

### Predicted vs actual values
The points in the scatter plot represent the predicted Wing_Span values (on the y-axis) against the actual Wing_Span values (on the x-axis) for the test set.

In [ ]:
print(y_test.shape)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Diagonal line
plt.title('Predicted vs Actual Wing Span')
plt.xlabel('Actual Wing Span')
plt.ylabel('Predicted Wing Span')
plt.xlim([y_test.min(), y_test.max()])
plt.ylim([y_test.min(), y_test.max()])
plt.grid()
plt.savefig('outputs/ml_pipeline/predict_analysis/predicted_vs_actual.png')  # Save the plot as a PNG file
plt.show()

### Residual plot

Calculating Residuals: The residuals are calculated by subtracting the predicted values (predictions) from the actual values (y_test). Residuals indicate the difference between what the model predicted and what the actual values were.

This histogram helps visualize the distribution of the residuals, which can indicate how well the model is performing. Ideally, residuals should be normally distributed around zero if the model is a good fit.

The Residuals Distribution shows that the distribution of errors (if negelecting the outlier) are such that all errors are all well within +- 5 meter. (and that the model rather tend overestimate than underestimate the wingspan).

In [ ]:
residuals = y_test - predictions
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.grid()
plt.savefig('outputs/ml_pipeline/predict_analysis/residuals_distribution.png')  # Save the plot as a PNG file
plt.show()

### Residuals vs Fitted plot

Each point represents a fitted value (predicted wingspan) from the train set plotted against its corresponding residual.

Ideally, the residuals should be randomly scattered around the horizontal line at zero without any discernible pattern however our pattern appear to be "high" which is consistent with our previous plots.

If we the "High pattern" is real it could indicate: it may indicate issues such as non-linearity, omitted variables, or model miss-specification.

We also notice an outlier at the bottom of the graf that we could hunt down and remove since it potentially could skew the data.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(predictions, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted')
plt.xlabel('Fitted Values (Predicted Wing Span)')
plt.ylabel('Residuals')
plt.grid()
plt.savefig('outputs/ml_pipeline/predict_analysis/residuals_vs_fitted.png')  # Save the plot
plt.show()


---

## Cross Validation with K-fold

Cross-validation gives a more reliable estimate of model performance and shows that our model is more reliable than we thought before having made the cross-validation (MSE before minus KSA after cross-validation) in MSE with 7.26.. - 4.41.. = 2,85..

In [ ]:
# Define the number of folds
# The selection of 10 folds is standard for more reliable estimates especially if the size of the data set allows which it does in our case.
#
n_folds = 10 

# Initialize KFold cross-validator
kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)

# Perform cross-validation and calculate MSE for each fold
cv_scores = cross_val_score(pipeline, X, y, cv=kf, scoring='neg_mean_squared_error')

# Since cross_val_score returns negative MSE, we convert it to positive
cv_scores = -cv_scores

mse_k_means = cv_scores.mean()

# Print out the cross-validation results
print(f'Cross-validation Mean Squared Errors for each fold: {cv_scores}')
print(f'Average MSE across {n_folds} folds: {mse_k_means}')

# Save MSE to a text file
with open('outputs/ml_pipeline/predict_analysis/mse_cv.txt', 'w') as f:
    f.write(f'Cross-validation Mean Squared Errors for each fold: {cv_scores}\n')
    f.write(f'Average MSE across {n_folds} folds: {cv_scores.mean()}\n')

## Evaluation of model prediction

All previous plots have appearntly given a slight overestimation of the Wingspan. This is confirmed by a 
Mean Error (ME) of approximately 5%. The Relative Error of 0.18% (based on a Wingspan range for the y_test of 29.08 feet) shows that this is error is not significant.

In [ ]:
# From now on we will use the result of the Cross-validation
mse = mse_k_means

# Mean Error (ME)
mean_error = np.mean(predictions - y_test)
print("Mean Error (ME):", mean_error)

# Mean Absolute Error (MAE)
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error (MAE):", mae)

# Mean value of y_test (used for relative error calculation)
mean_y_test = np.mean(y_test)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Relative Error (using RMSE)
relative_error_rmse = rmse / mean_y_test
print(f"Relative Error (using RMSE): {relative_error_rmse:.4f} ({relative_error_rmse * 100:.2f}%)")

# R-Squared (R²)
y_mean = np.mean(y_test)
ss_total = np.sum((y_test - y_mean) ** 2)
ss_residual = np.sum((y_test - predictions) ** 2)
r_squared = 1 - (ss_residual / ss_total)
print("R-Squared (R²):", r_squared)

# Save the results to a text file
output_path = 'outputs/ml_pipeline/predict_analysis/error_analysis.txt'
with open(output_path, 'w') as f:
    f.write(f'Mean Error (ME): {mean_error:.4f} feet\n')
    f.write(f'Mean Absolute Error (MAE): {mae:.4f} feet\n')
    f.write(f'Mean Squared Error (MSE): {mse:.4f} feet\n')
    f.write(f'Root Mean Squared Error (RMSE): {rmse:.4f} feet\n')
    f.write(f'Relative Error using RMSE (RE)): {relative_error_rmse:.4f} ({relative_error_rmse * 100:.2f}%)\n')
    f.write(f'R-Squared (R²): {r_squared:.4f}\n')

Code for generating confusion matrix where the numerical continuous values has been binned into segments. Not that the code is cleaning/dropping NaN (due to no data points in those "bucket"-intervals) that, if they would not be removed, would break the function.

In [ ]:
def create_confusion_matrix(y_test, predictions, bins, bin_labels):
    """
    Creates and visualizes both a normalized and raw confusion matrix by discretizing
    the continuous target and prediction values into bins.
    
    Parameters:
    -----------
    y_test : array-like
        The true values of the target variable (e.g., `Wing_Span`).
        
    predictions : array-like
        The predicted values of the target variable (e.g., predicted `Wing_Span`).
        
    bins : list
        A list of bin edges used to discretize the continuous values.
        
    bin_labels : list
        A list of labels for each bin, corresponding to the bin edges.
        
    Returns:
    --------
    None
    """
    
    # Discretize the actual values and predictions into the bins (using the bin edges)
    y_test_binned = pd.cut(y_test, bins=bins, labels=False, right=False)  # right=False makes bins left-inclusive
    predictions_binned = pd.cut(predictions, bins=bins, labels=False, right=False)
    
    # Calculate confusion matrix using numerical indices for bins
    cm = confusion_matrix(y_test_binned, predictions_binned)
    
    # Normalize the confusion matrix (by row, i.e., actual values)
    cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize by row (actual values)
    
    # Create a figure with two subplots
    plt.figure(figsize=(12, 6))

    # Raw confusion matrix plot (without normalization)
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=bin_labels, yticklabels=bin_labels)
    plt.title('Raw Confusion Matrix')
    plt.xlabel('Predicted Wing Span')
    plt.ylabel('Actual Wing Span')

    # Normalized confusion matrix plot (percentage format)
    plt.subplot(1, 2, 2)
    sns.heatmap(cm_percentage, annot=True, fmt='.2%', cmap='Blues', xticklabels=bin_labels, yticklabels=bin_labels)
    plt.title('Normalized Confusion Matrix')
    plt.xlabel('Predicted Wing Span')
    plt.ylabel('Actual Wing Span')

    # Save the confusion matrix plot (both raw and normalized)
    plt.savefig("outputs/ml_pipeline/predict_analysis/confusion_matrix.png")

    # Adjust layout and show plot
    plt.tight_layout()
    plt.show()

    return cm, cm_percentage

def clean_data(y_test, predictions):
    """
    Cleans the input data by removing NaN or infinite values.
    
    Parameters:
    -----------
    y_test : array-like
        The true values of the target variable (e.g., `Wing_Span`).
        
    predictions : array-like
        The predicted values of the target variable (e.g., predicted `Wing_Span`).
        
    Returns:
    --------
    cleaned_y_test : array-like
        Cleaned true values of the target variable (no NaN or infinite values).
        
    cleaned_predictions : array-like
        Cleaned predicted values (no NaN or infinite values).
    """
    # Remove NaN or infinite values
    mask = np.isfinite(y_test) & np.isfinite(predictions)
    cleaned_y_test = y_test[mask]
    cleaned_predictions = predictions[mask]
    
    return cleaned_y_test, cleaned_predictions

# Example usage of the function:
min_wing_span = 16
max_wing_span = 104

# Create 5 equally spaced bins between the minimum and maximum values
bins_5 = np.linspace(min_wing_span, max_wing_span, 6)  # 6 edges = 5 bins
print("5 Bins Edges:", bins_5)

# Create bin labels based on the bin edges
bin_labels_5 = [f'{round(bins_5[i], 1)} to {round(bins_5[i+1], 1)}' for i in range(len(bins_5)-1)]
print("Bin Labels:", bin_labels_5)

# Check if y_test or predictions contain NaN or Inf values
print("Checking for NaN or Inf values in y_test:")
print(f"y_test contains NaN: {np.any(np.isnan(y_test))}")
print(f"y_test contains Inf: {np.any(np.isinf(y_test))}")

print("Checking for NaN or Inf values in predictions:")
print(f"predictions contains NaN: {np.any(np.isnan(predictions))}")
print(f"predictions contains Inf: {np.any(np.isinf(predictions))}")

# If any NaN or Inf values found, remove or replace them
valid_mask = ~np.isnan(y_test) & ~np.isinf(y_test) & ~np.isnan(predictions) & ~np.isinf(predictions)
y_test_clean = y_test[valid_mask]
predictions_clean = predictions[valid_mask]

# Call the function with the actual and predicted values
confusion_matrix_result, normalized_confusion_matrix = create_confusion_matrix(y_test_clean, predictions_clean, bins_5, bin_labels_5)

---

# Push files to the repo

The following plots has already been save upstream right after they where created:
* feature_importance (3 files) 
* Mean Squared Error
* Predicted vs Actual Plot
* Residuals Distribution Plot
* Residuals vs Fitted Plot

Below we will generate the following files:
* Train set
* Test set
* Modeling pipeline

In [ ]:
joblib.dump(pipeline, 'outputs/ml_pipeline/predict_analysis/wingspan_predictor_model.pkl')


In [ ]:
file_path = f'outputs/ml_pipeline/predict_analysis'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

## Train Set: features and target

In [ ]:
X_train.head()

In [24]:
X_train.to_csv('outputs/ml_pipeline/predict_analysis/X_train.csv', index=False)

In [25]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [27]:
y_train.to_csv('outputs/ml_pipeline/predict_analysis/y_train.csv', index=False)

## Test Set: features and target

In [ ]:
X_test.head()

In [29]:
X_test.to_csv('outputs/ml_pipeline/predict_analysis/X_test_head.csv', index=False)

In [ ]:
y_test

In [31]:
y_test.to_csv('outputs/ml_pipeline/predict_analysis/y_test.csv', index=False)

---